In [ ]:
import os
os.cpu_count()

In [1]:
import sqlalchemy
import logging


In [14]:
from testcontainers.core.container import DockerContainer
from testcontainers.core.waiting_utils import wait_for_logs
dc = DockerContainer("ghcr.io/internetofwater/nldi-db:demo")
dc.with_exposed_ports(5432)
dc.with_env("POSTGRES_PASSWORD", "changeMe")
dc.with_env("NLDI_DATABASE_ADDRESS", "localhost")
dc.with_env("NLDI_DATABASE_NAME", "nldi")
dc.with_env("NLDI_DB_OWNER_USERNAME", "nldi")
dc.with_env("NLDI_DB_OWNER_PASSWORD", "changeMe")
dc.with_env("NLDI_SCHEMA_OWNER_USERNAME", "nldi_schema_owner")
dc.with_env("NLDI_SCHEMA_OWNER_PASSWORD", "changeMe")
dc.with_env("NHDPLUS_SCHEMA_OWNER_USERNAME", "nhdplus")
dc.with_env("NLDI_READ_ONLY_USERNAME", "read_only_user")
dc.with_env("NLDI_READ_ONLY_PASSWORD", "changeMe")
dc.with_volume_mapping("data", "/var/lib/postgresql/data", mode="rw")

dc.start()
#delay = wait_for_logs(dc, "database system is ready to accept connections")
#logging.info(f"Postgres container started in {delay} seconds")

client = dc.get_docker_client()
db_info = {
    "user": "nldi",
    "password": "changeMe",
    "host": client.bridge_ip(dc.get_wrapped_container().short_id),
    "port": 5432,
    "dbname": "nldi",
}




Pulling image ghcr.io/internetofwater/nldi-db:demo


Container started: 488b7658ec16


In [20]:
client = dc.get_docker_client()
print(f"{client.bridge_ip(dc.get_wrapped_container().short_id)}:{dc.get_exposed_port(5432)}")

Waiting for container <Container: 488b7658ec16> with image ghcr.io/internetofwater/nldi-db:demo to be ready ...


172.17.0.2:32769


In [19]:

stdout, stderr = dc.get_logs()
print(stdout.decode())





The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locale "en_US.utf8".
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "english".

Data page checksums are disabled.

fixing permissions on existing directory /var/lib/postgresql/data ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... Etc/UTC
creating configuration files ... ok
running bootstrap script ... ok
performing post-bootstrap initialization ... ok
syncing data to disk ... ok


Success. You can now start the database server using:

    pg_ctl -D /var/lib/postgresql/data -l logfile start

waiting for server to start....2024-08-05 14:34:25.102 UTC [46] LOG:  starting PostgreSQL 12.5 (Debi

In [ ]:






dc.stop()

In [21]:
engine = sqlalchemy.create_engine(f'postgresql+psycopg2://nldi:changeMe@{client.bridge_ip(dc.get_wrapped_container().short_id)}:5432/nldi')
with engine.begin() as connection:
    v, = connection.execute(sqlalchemy.text("SELECT 1")).fetchone()
print(v)

1


In [22]:
dc.stop()